## Import Libraries

In [68]:
from pyomo.environ import *
import highspy

## Declare the Variables

In [69]:
Sector = ['Pertanian', 'Manufaktur', 'Jasa', 'Infrastruktur']   # Sector names
S = ['A', 'M', 'S', 'I']                                   # Sector codes

In [70]:
a = {  # Input-Output coefficients
    'A': {'A': 0.20, 'M': 0.10, 'S': 0.05, 'I': 0.08},
    'M': {'A': 0.05, 'M': 0.25, 'S': 0.10, 'I': 0.12},
    'S': {'A': 0.02, 'M': 0.06, 'S': 0.15, 'I': 0.05},
    'I': {'A': 0.01, 'M': 0.04, 'S': 0.02, 'I': 0.18},
}

In [71]:
d = {'A': 50, 'M': 75, 'S': 50, 'I': 40}        # Minimum demand
va = {'A': 0.6, 'M': 0.8, 'S': 0.75, 'I': 0.9}  # Value added coefficients

In [72]:
# Resource usage coefficients
labour = {'A': 2, 'M': 3, 'S': 1, 'I': 3}
capital = {'A': 1.1, 'M': 2, 'S': 0.8, 'I': 2.5}
environment = {'A': 0.5, 'M': 0.9, 'S': 0.3, 'I': 1.1}

# Resource availability
L_max = 1000
C_max = 1200 
E_max = 400

## Declare Model

In [73]:
m = ConcreteModel()
m.I = Set(initialize=S)
m.p = Var(m.I, domain=NonNegativeReals)

## Set Constraints

In [74]:
# Ensure minimum demand is met considering input-output relationships
def io_rule(m, j):
    return sum(a[i][j] * m.p[j] for i in m.I) + d[j] <= m.p[j]
m.io = Constraint(m.I, rule=io_rule)

In [75]:
# Multiply resource and production, then sum all results, ensure it is less than max available resource
m.labour = Constraint(expr = sum(labour[i] * m.p[i] for i in m.I) <= L_max)
m.capital = Constraint(expr = sum(capital[i] * m.p[i] for i in m.I) <= C_max)
m.environment = Constraint(expr = sum(environment[i] * m.p[i] for i in m.I) <= E_max)
m.fiscal = Constraint(expr = m.p['I'] <= 120)

## Set Objective

In [76]:
# For each i in m.I, multiply value added and production, then sum all results
m.obj = Objective(expr = sum(va[i] * m.p[i] for i in m.I), sense=maximize)

## Solve Model

In [77]:
solver = SolverFactory('highs')
results = solver.solve(m)

## Show Results

In [78]:
print("PDB per Sektor:")
for i, sector in enumerate(m.I):
    print(f"{Sector[i]:14} = {va[sector]*m.p[sector]():>6.2f} ({va[sector]*m.p[sector]()/m.obj()*100:>5.2f}%)")
print(f"TOTAL          = {m.obj():.2f} Unit")

print("\nPenggunaan Sumber Daya:")
print(f"Tenaga Kerja : {sum(labour[i]*m.p[i]() for i in m.I):.2f}/1000")
print(f"Modal        :  {sum(capital[i]*m.p[i]() for i in m.I):.2f}/1200")
print(f"Batas Emisi  :  {sum(environment[i]*m.p[i]() for i in m.I):.2f}/400")

PDB per Sektor:
Pertanian      =  41.67 (10.55%)
Manufaktur     = 109.09 (27.62%)
Jasa           = 181.12 (45.85%)
Infrastruktur  =  63.16 (15.99%)
TOTAL          = 395.04 Unit

Penggunaan Sumber Daya:
Tenaga Kerja : 1000.00/1000
Modal        :  717.75/1200
Batas Emisi  :  307.09/400


## Conclusion

1. Setelah melakukan optimisasi menggunakan solver, 395.04 merupakan nilai PDB tertinggi yang didapatkan berdasarkan ketentuan-ketentuan yang diterapkan
2. Kontributor PDB terbesar adalah Jasa(46%), diikuti oleh Manufaktur(28%). Berikut implikasi yang bisa didapatkan
   - Pemerintah sebaiknya memperkuat sektor jasa (pendidikan, keuangan, pariwisata, digital economy) karena:
     - Efisien dalam penggunaan tenaga kerja dan modal.
     - Memberi kontribusi terbesar terhadap pertumbuhan ekonomi.
   - Manufaktur masih penting untuk kestabilan ekonomi jangka panjang karena efek pengganda ke sektor lain (supply chain, ekspor).
   - Pertanian relatif kecil kontribusinya, tapi tetap vital bagi ketahanan pangan dan kesejahteraan pedesaan.
   - Infrastruktur memberi dampak tidak langsung (meningkatkan efisiensi sektor lain), walau kontribusinya moderat secara langsung.
3. Adanya pembatasan pada tenanga kerja dan modal, tentu berpengaruh besar pada PDB yang akan dihasilkan. Karena adanya pembatasan, maka modal dan tenaga kerja lebih banyak dialokasikan ke dalam sektor yang efisien dalam penggunaan sumber daya, yakni sektor jasa. Jika dilakukan penambahan tenaga kerja dan modal, maka PDM akan semangkin meningkat, dan bisa dilakukan investasi lebih besar pada sektor yang butuh investasi tinggi.